In [1]:
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Jupyter")
AddReference("QuantConnect.Indicators")
from System import *
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Jupyter import *
from QuantConnect.Indicators import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd

def initial_system():
    qb = QuantBook()

    volatility_etfs = ["SQQQ", "TQQQ", "TVIX", "VIXY", "SPLV",
                   "UVXY", "EEMV", "EFAV", "USMV","SPY"]
    # Add Equity ------------------------------------------------
    for i in range(len(volatility_etfs)):
        qb.AddEquity(volatility_etfs[i],Resolution.Daily)
    history = qb.History(qb.Securities.Keys, 252*15,Resolution.Daily)
    return qb,history

ModuleNotFoundError: No module named 'clr'

In [ ]:
qb,history = initial_system()[0],initial_system()[1]

#先把收盤價換成成log
close = history.loc[qb.Securities.Keys[0]]['close']
close_log = np.log(close)

#處理 At*Ax = At*b
identity = pd.DataFrame(np.ones((len(close_log.index),1)),index = close_log.index)
another = pd.DataFrame([x for x in range(1,len(close_log.index)+1,1)],index = close_log.index)
matrix = pd.concat((another,identity),axis = 1)
close_log = np.array(close_log)

#利用反矩陣算出最佳解
matrix = np.array(matrix)
matrix_I = np.linalg.inv((matrix.T).dot(matrix)) 
Answer = (matrix_I.dot(matrix.T)).dot(close_log)

#劃出預估圖跟原始資料對比，來預測收盤價是否有吻合
Monitor_price = [np.exp(Answer[0]*x+Answer[1]) for x in range(1,len(close.index)+1,1)]
Monitor_price = pd.DataFrame(Monitor_price,index = close.index,columns=['Pre_close'])
pd.concat([Monitor_price,close],axis = 1).plot()